<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/5%EC%9D%BC%EC%83%81%EC%8A%B9%EC%A2%85%EB%AA%A9%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests konlpy
!pip install Article
!pip install pykrx
!pip install pandas_market_calendars
!pip install feedparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 25.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement Article (from versions: none)
ERROR: No matching distribution found for Article
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
  Created wheel for 

# 최근 5거래일 동안 20% 이상 상승한 종목의 1) 상승율, 시가총액 2) 뉴스 3) 검색량 증가율

In [2]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import requests
import re
import json
import pandas as pd
import pytz
import calendar
from IPython.display import Markdown, display
import feedparser

# 한국 시간(Asia/Seoul) 설정
seoul_tz = pytz.timezone("Asia/Seoul")

# 1) 최근 5거래일 중 20% 이상 상승한 종목의 상승률과 시가총액을 보여준다.
def get_krx_trading_range():
    krx = mcal.get_calendar('XKRX')
    today = datetime.now(seoul_tz).date()
    start_date = today - timedelta(days=30)
    trading_days = krx.valid_days(start_date, today)
    # 최근 5거래일 중 첫 번째(5일 전)와 마지막(최신 거래일)을 구함
    start_trading_day = trading_days[-5]
    last_trading_day = trading_days[-1]
    return start_trading_day.date(), last_trading_day.date()

start_date, end_date = get_krx_trading_range()
print(f"시작일: {start_date}, 종료일: {end_date}")

# 주가 변동 데이터 가져오기
df = stock.get_market_price_change(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"), "ALL")

# 등락률이 20% 이상인 종목 필터링 및 내림차순 정렬
rising_stocks = df[df['등락률'] >= 20].sort_values('등락률', ascending=False)

# 시가총액 정보 가져오기
market_cap = stock.get_market_cap(end_date.strftime("%Y%m%d"))

# 결과 출력 및 종목명 리스트 생성 (종목명(티커) 형식)
print("최근 5일간 20% 이상 상승한 종목:")
stock_names = []
name_to_ticker = {}  # 종목명과 티커 매핑
for ticker in rising_stocks.index:
    stock_name = stock.get_market_ticker_name(ticker)
    name_to_ticker[stock_name] = ticker
    change_rate = rising_stocks.loc[ticker, '등락률']
    cap = market_cap.loc[ticker, '시가총액']
    cap_billion = cap / 100_000_000  # 억원 단위로 변환
    print(f"{stock_name}({ticker}) (등락률: {change_rate:.2f}%, 시가총액: {cap_billion:.2f}억원)")
    stock_names.append(stock_name)

# 네이버 API 정보 (발급받은 값 입력)
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 날짜 설정 (트렌드 데이터: 최근 30일)
END_DATE = datetime.now(seoul_tz)
START_DATE = END_DATE - timedelta(days=30)

def get_trend_data(keywords):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
        "Content-Type": "application/json"
    }
    # 최대 5개씩 나누어 API 호출
    chunk_size = 5
    keyword_chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]
    all_results = []
    for chunk in keyword_chunks:
        body = {
            "startDate": START_DATE.strftime("%Y-%m-%d"),
            "endDate": END_DATE.strftime("%Y-%m-%d"),
            "timeUnit": "date",
            "keywordGroups": [{"groupName": keyword, "keywords": [keyword]} for keyword in chunk]
        }
        response = requests.post(url, headers=headers, data=json.dumps(body))
        data = json.loads(response.text)
        if 'results' in data:
            all_results.extend(data["results"])
        else:
            print("Error: 'results' key not found in API response for chunk:", chunk)
            print("API Response:", response.text)
    return all_results

def analyze_trends(trend_data):
    df_list = []
    for item in trend_data:
        df = pd.DataFrame(item["data"])
        df["keyword"] = item["title"]
        df_list.append(df)
    if not df_list:
        print("Error: No data to analyze. Check the API response and parameters.")
        return pd.DataFrame()
    df_combined = pd.concat(df_list)
    df_pivot = df_combined.pivot(index="period", columns="keyword", values="ratio")
    # 최근 7일과 이전 7일의 평균 검색량 비교
    recent_7days = df_pivot.iloc[-7:].mean()
    previous_7days = df_pivot.iloc[-14:-7].mean()
    growth_rate = (recent_7days - previous_7days) / previous_7days * 100
    return growth_rate.sort_values(ascending=False)

# 트렌드 데이터 가져오기 및 분석
trend_data = get_trend_data(stock_names)
growth_rates = analyze_trends(trend_data)

print("\n검색량 증가율 (%):")
for keyword, rate in growth_rates.items():
    ticker = name_to_ticker.get(keyword, "N/A")
    print(f"{keyword}({ticker}): {rate:.2f}%")

# 상승종목의 뉴스 가져오기 (최근 7일)
news_end_date = datetime.now(seoul_tz)
news_start_date = news_end_date - timedelta(days=7)

headers_news = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}

def clean_text(text):
    text = text.replace("&amp;", "&")
    text = text.replace("quot;", '')
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&sort=date&display=10"
    response = requests.get(url, headers=headers_news)
    if response.status_code == 200:
        news_data = response.json()
        print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
        count = 0
        for item in news_data["items"]:
            # pubDate를 %z 포맷을 사용해 파싱 후 서울 시간으로 변환
            pub_date = datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S %z")
            pub_date = pub_date.astimezone(seoul_tz)
            if news_start_date <= pub_date <= news_end_date:
                count += 1
                title = clean_text(item["title"])
                link = item["link"]
                print(f"{count}. {pub_date.strftime('%Y-%m-%d')} [{title}]({link})")
        if count == 0:
            print("최근 7일간 관련 뉴스가 없습니다.")
    else:
        print("네이버 API 호출 실패:", response.status_code)

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    # 구글 뉴스 RSS 피드 URL (검색어: 한국무브넥스)
    rss_url = f"https://news.google.com/rss/search?q={query}&{ticker}&hl=ko&gl=KR&ceid=KR:ko"

    # RSS 피드 파싱
    feed = feedparser.parse(rss_url)

    # 현재 시간과 7일 전 날짜 계산 (한국 시간 기준)
    now = datetime.now(seoul_tz)
    cutoff = now - timedelta(days=7)

    news_items = []

    # RSS 피드의 각 항목을 확인하여 지난 7일 이내의 뉴스만 선택 (최대 10개)
    for entry in feed.entries:
        # published_parsed는 time.struct_time 형식이므로 calendar.timegm을 사용해 timestamp로 변환
        ts = calendar.timegm(entry.published_parsed)
        published_dt = datetime.fromtimestamp(ts, seoul_tz)
        if published_dt >= cutoff:
            news_items.append((entry.title, entry.link))
        if len(news_items) >= 10:
            break

    # # 결과 출력 (헤드라인과 링크)
    # print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
    # for idx, (title, link) in enumerate(news_items, start=1):
    #     print(f"{idx}. {title}\n   링크: {link}")
    # 결과를 Markdown 형식으로 생성하여 출력 (제목 클릭 시 링크로 연결, 링크는 노출되지 않음)
    markdown_text = f"\n최근 7일간 '{query}({ticker})' 관련 뉴스: (최대 10개):\n\n"
    for idx, (title, link) in enumerate(news_items, start=1):
        markdown_text += f"{idx}. [{title}]({link})\n\n"

    display(Markdown(markdown_text))


/usr/local/lib/python3.11/dist-packages/pandas_market_calendars/market_calendar.py:179: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(


시작일: 2025-01-23, 종료일: 2025-02-04
최근 5일간 20% 이상 상승한 종목:
에스에이티이엔지(351320) (등락률: 95.00%, 시가총액: 601.03억원)
엔솔바이오사이언스(140610) (등락률: 65.00%, 시가총액: 3787.74억원)
한국무브넥스(010100) (등락률: 59.90%, 시가총액: 1881.84억원)
플리토(300080) (등락률: 52.63%, 시가총액: 1386.46억원)
하이젠알앤엠(160190) (등락률: 51.90%, 시가총액: 12972.96억원)
루트락(253610) (등락률: 50.00%, 시가총액: 142.00억원)
GRT(900290) (등락률: 48.87%, 시가총액: 4002.07억원)
바이브컴퍼니(301300) (등락률: 46.83%, 시가총액: 746.17억원)
에이원알폼(234070) (등락률: 46.76%, 시가총액: 103.87억원)
디젠스(113810) (등락률: 44.31%, 시가총액: 579.15억원)
로지스몬(223220) (등락률: 42.93%, 시가총액: 130.32억원)
온코크로스(382150) (등락률: 42.86%, 시가총액: 1152.55억원)
진양화학(051630) (등락률: 42.30%, 시가총액: 1048.34억원)
파워풀엑스(266870) (등락률: 40.25%, 시가총액: 35.95억원)
유니온(000910) (등락률: 34.94%, 시가총액: 1091.25억원)
레인보우로보틱스(277810) (등락률: 34.39%, 시가총액: 65959.52억원)
카페24(042000) (등락률: 33.99%, 시가총액: 13145.16억원)
인스웨이브시스템즈(450520) (등락률: 32.13%, 시가총액: 752.87억원)
대한광통신(010170) (등락률: 31.70%, 시가총액: 798.76억원)
라온테크(232680) (등락률: 31.59%, 시가총액: 1138.11억원)
비투엔(307870) (등락률: 29.65%, 시가총액: 613.23억원)
KS인더스트리

NameError: name 'feedparser' is not defined

#5일간 20% 이상 상승한 종목의 60일간 주가데이터를 받아옴


In [ ]:
!pip install pykrx

In [ ]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    final_df = pd.DataFrame()
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        # print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df = df[['종가']].rename(columns={'종가': company_name})


        # 최고가/최저가 계산
        max_prices[company_name] = df[company_name].max()
        min_prices[company_name] = df[company_name].min()
        max_rates[company_name] = round(((df[company_name].max()/df[company_name].iloc[0])-1)*100,2)
        min_rates[company_name] = round(((df[company_name].min()/df[company_name].iloc[0])-1)*100,2)
        max_dates[company_name] = df[company_name].idxmax().strftime('%Y-%m-%d')
        min_dates[company_name] = df[company_name].idxmin().strftime('%Y-%m-%d')
        # print(df[company_name][0])

        # 데이터 병합
        final_df = pd.merge(final_df, df, left_index=True, right_index=True, how='outer') if not final_df.empty else df
        # print(final_df)
    # 추가 행 생성 (수정)
    additional_data = {
    '종목별': ['최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일'],
    **{name: [
        min_prices.get(name, 'N/A'),
        max_prices.get(name, 'N/A'),
        min_rates.get(name, 'N/A'),
        max_rates.get(name, 'N/A'),
        min_dates.get(name, 'N/A'),
        max_dates.get(name, 'N/A')
    ] for name in company_names}
}
    additional_df = pd.DataFrame(additional_data).set_index('종목별')

    return pd.concat([final_df, additional_df])

# # 입력 데이터
# stocks = [
#     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
#     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
#     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
#     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
#     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
# ]



# 엑셀 파일 읽기
df = pd.read_excel("/종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })


result_df = get_stock_data_pivot(stocks)
result_df.to_excel("stock_analysis_final.xlsx")
print(result_df)


0 위메이드_0
1 RFHIC_1
2 야스_2
3 두산_3
4 코오롱티슈진_4
5 브릿지바이오테라퓨틱스_5
6 코스모화학_6
7 롯데케미칼_7
8 제이앤티씨_8
9 대한유화_9
10 효성티앤씨_10
11 송원산업_11
12 코스모신소재_12
13 대주전자재료_13
14 엘앤에프_14
15 LS_15
16 한진칼_16
17 바이오다인_17
18 원준_18
19 HPSP_19
20 에이스테크_20
21 HD현대일렉트릭_21
22 영풍정밀_22
23 고려아연_23
24 유한양행_24
25 레인보우로보틱스_25
26 HD현대에너지솔루션_26
27 레이크머티리얼즈_27
28 티앤엘_28
29 솔브레인홀딩스_29
30 엘티씨_30
31 SK하이닉스_31
32 엔켐_32
33 인탑스_33
34 이엔에프테크놀로지_34
35 GST_35
36 월덱스_36
37 한화솔루션_37
38 SBI핀테크솔루션즈_38
39 디앤씨미디어_39
40 주성엔지니어링_40
41 제우스_41
42 이수페타시스_42
43 휴젤_43
44 제넥신_44
45 리노공업_45
46 크라우드웍스_46
47 ISC_47
48 SK오션플랜트_48
49 녹십자_49
50 뷰노_50
51 휴메딕스_51
52 SK바이오팜_52
53 씨에스윈드_53
54 파마리서치_54
55 헬릭스미스_55
56 이엠텍_56
57 태영건설_57
58 HD현대중공업_58
59 올릭스_59
60 F&F_60
61 성광벤드_61
62 GS건설_62
63 태광_63
64 HD현대건설기계_64
65 다산네트웍스_65
66 코오롱티슈진_66
67 두산로보틱스_67
68 효성중공업_68
69 브릿지바이오테라퓨틱스_69
70 케이씨텍_70
71 유니테크노_71
72 푸드나무_72
73 삼천당제약_73
74 넥슨게임즈_74
75 솔루스첨단소재_75
76 이랜텍_76
77 더존비즈온_77
78 메디톡스_78
79 코웨이_79
80 세진중공업_80
81 GS_81
82 DB하이텍_82
83 LIG넥스원_83
84 코스맥스_84
85 화신_85
86 서연

#엑셀자료 json변환

In [ ]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_excel("/종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
print(df)
# DataFrame을 JSON 형식으로 변환
json_data = []
i = 0
for _, row in df.iterrows():
    json_data.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })
    i += 1

# JSON 형식으로 출력
print("[")
for i, item in enumerate(json_data):
    if i < len(json_data) - 1:
        print(f"    {str(item)},")
    else:
        print(f"    {str(item)}")
print("]")

          종목명        매수일        매도일     종목코드       수익률
0        위메이드 2024-10-30 2025-01-31  A112040  0.025641
1       RFHIC 2024-10-25 2025-01-22  A218410  0.029903
2          야스 2024-10-24 2025-01-21  A255440 -0.203985
3          두산 2024-10-18 2025-01-15  A000150  0.330097
4      코오롱티슈진 2024-10-17 2025-01-14  A950160  0.848559
...       ...        ...        ...      ...       ...
1809      제우스 2017-01-06 2017-04-05  A079370  0.087818
1810      HLB 2017-01-05 2017-04-04  A028300  0.003252
1811  뉴파워프라즈마 2017-01-02 2017-03-30  A144960 -0.004357
1812   대주전자재료 2017-01-02 2017-03-30  A078600  0.047674
1813    동아에스티 2017-01-02 2017-03-30  A170900 -0.105266

[1814 rows x 5 columns]
[
    {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
    {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
    {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
    {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
    {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},

#종목명 체크

In [ ]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    # final_df = pd.DataFrame()
    # max_prices = {}
    # min_prices = {}
    # max_rates = {}
    # min_rates = {}
    # max_dates = {}
    # min_dates = {}
    # # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    # company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        # company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        # print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {i} {stock_info['종목명']}")
            continue

    return


# 엑셀 파일 읽기
df = pd.read_excel("종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })

# print(stocks)

get_stock_data_pivot(stocks)
# result_df.to_excel("stock_analysis_final.xlsx")
# print(result_df)


종목을 찾을 수 없습니다: 870 상장폐지
종목을 찾을 수 없습니다: 970 상장폐지
종목을 찾을 수 없습니다: 1016 LX 하우시스
종목을 찾을 수 없습니다: 1219 상장폐지


# 인덱스 리셋

In [ ]:
!pip install pykrx

In [ ]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    final_df = pd.DataFrame()
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df = df[['종가']].rename(columns={'종가': company_name})
        df.reset_index(drop=True, inplace=True)

        # 최고가/최저가 계산
        max_prices[company_name] = df[company_name].max()
        min_prices[company_name] = df[company_name].min()
        max_rates[company_name] = round(((df[company_name].max()/df[company_name].iloc[0])-1)*100,2)
        min_rates[company_name] = round(((df[company_name].min()/df[company_name].iloc[0])-1)*100,2)
        max_dates[company_name] = df[company_name].idxmax()
        min_dates[company_name] = df[company_name].idxmin()
        # print(df[company_name][0])

        # 데이터 병합
        final_df = pd.merge(final_df, df, left_index=True, right_index=True, how='outer') if not final_df.empty else df
        # print(final_df)
    # 추가 행 생성 (수정)
    additional_data = {
    '종목별': ['최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일'],
    **{name: [
        min_prices.get(name, 'N/A'),
        max_prices.get(name, 'N/A'),
        min_rates.get(name, 'N/A'),
        max_rates.get(name, 'N/A'),
        min_dates.get(name, 'N/A'),
        max_dates.get(name, 'N/A')
    ] for name in company_names}
}
    additional_df = pd.DataFrame(additional_data).set_index('종목별')

    return pd.concat([final_df, additional_df])

# # 입력 데이터
# stocks = [
#     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
#     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
#     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
#     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
#     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
# ]



# 엑셀 파일 읽기
df = pd.read_excel("종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })


result_df = get_stock_data_pivot(stocks)
result_df.to_excel("stock_analysis_final.xlsx")
print(result_df)


0 위메이드_0
1 RFHIC_1
2 야스_2
3 두산_3
4 코오롱티슈진_4
5 브릿지바이오테라퓨틱스_5
6 코스모화학_6
7 롯데케미칼_7
8 제이앤티씨_8
9 대한유화_9
10 효성티앤씨_10
11 송원산업_11
12 코스모신소재_12
13 대주전자재료_13
14 엘앤에프_14
15 LS_15
16 한진칼_16
17 바이오다인_17
18 원준_18
19 HPSP_19
20 에이스테크_20
21 HD현대일렉트릭_21
22 영풍정밀_22
23 고려아연_23
24 유한양행_24
25 레인보우로보틱스_25
26 HD현대에너지솔루션_26
27 레이크머티리얼즈_27
28 티앤엘_28
29 솔브레인홀딩스_29
30 엘티씨_30
31 SK하이닉스_31
32 엔켐_32
33 인탑스_33
34 이엔에프테크놀로지_34
35 GST_35
36 월덱스_36
37 한화솔루션_37
38 SBI핀테크솔루션즈_38
39 디앤씨미디어_39
40 주성엔지니어링_40
41 제우스_41
42 이수페타시스_42
43 휴젤_43
44 제넥신_44
45 리노공업_45
46 크라우드웍스_46
47 ISC_47
48 SK오션플랜트_48
49 녹십자_49
50 뷰노_50
51 휴메딕스_51
52 SK바이오팜_52
53 씨에스윈드_53
54 파마리서치_54
55 헬릭스미스_55
56 이엠텍_56
57 태영건설_57
58 HD현대중공업_58
59 올릭스_59
60 F&F_60
61 성광벤드_61
62 GS건설_62
63 태광_63
64 HD현대건설기계_64
65 다산네트웍스_65
66 코오롱티슈진_66
67 두산로보틱스_67
68 효성중공업_68
69 브릿지바이오테라퓨틱스_69
70 케이씨텍_70
71 유니테크노_71
72 푸드나무_72
73 삼천당제약_73
74 넥슨게임즈_74
75 솔루스첨단소재_75
76 이랜텍_76
77 더존비즈온_77
78 메디톡스_78
79 코웨이_79
80 세진중공업_80
81 GS_81
82 DB하이텍_82
83 LIG넥스원_83
84 코스맥스_84
85 화신_85
86 서연

#종목별 최종, 최고, 최저만 기록

In [3]:
!pip install pykrx

In [21]:
import pandas as pd
from pykrx import stock
from datetime import datetime

time = datetime.now(seoul_tz)
print(time)

def get_stock_data_summary(stock_list):
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    final_rates = {}
    start_prices = {}
    end_prices = {}
    company_names = []

    for i, stock_info in enumerate(stock_list[:]):
        company_name = f"{stock_info['종목명']}_{i}"
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        print(i, company_name)

        time1 = datetime.now(seoul_tz)
        print(time1)

        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)
        print(ticker)
        time2 = datetime.now(seoul_tz)
        print(time2)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df.reset_index(drop=True, inplace=True)
        print(df)
        time3 = datetime.now(seoul_tz)
        print(time3)

        max_prices[company_name] = df['종가'].max()
        min_prices[company_name] = df['종가'].min()
        max_rates[company_name] = round(((df['종가'].max()/df['종가'].iloc[0])-1)*100, 2)
        min_rates[company_name] = round(((df['종가'].min()/df['종가'].iloc[0])-1)*100, 2)
        max_dates[company_name] = df['종가'].idxmax()
        min_dates[company_name] = df['종가'].idxmin()
        final_rates[company_name] = round(((df['종가'].iloc[-1]/df['종가'].iloc[0])-1)*100, 2)
        start_prices[company_name] = df['종가'].iloc[0]
        end_prices[company_name] = df['종가'].iloc[-1]


    additional_data = {
        '종목별': ['시작일주가', '종료일주가', '최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일', '최종수익률'],
        **{name: [
            start_prices.get(name, 'N/A'),
            end_prices.get(name, 'N/A'),
            min_prices.get(name, 'N/A'),
            max_prices.get(name, 'N/A'),
            min_rates.get(name, 'N/A'),
            max_rates.get(name, 'N/A'),
            min_dates.get(name, 'N/A'),
            max_dates.get(name, 'N/A'),
            final_rates.get(name, 'N/A'),
        ] for name in company_names}
    }
    print(additional_data)
    time4 = datetime.now(seoul_tz)
    print(time4)
    return pd.DataFrame(additional_data).set_index('종목별')

# 입력 데이터
stocks = [
    {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
    {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
    {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
    {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
    {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
]


# # 엑셀 파일 읽기
# df = pd.read_excel("종목리스트_5일_20%.xlsx", "Sheet2")
# stocks = []
# for _, row in df.iterrows():
#     stocks.append({
#         "종목명": row['종목명'],
#         "매수일": row['매수일'].strftime('%Y-%m-%d'),
#         "매도일": row['매도일'].strftime('%Y-%m-%d')
#     })

result_df = get_stock_data_summary(stocks)
print(result_df)
time5 = datetime.now(seoul_tz)
print(time5)
result_df.to_excel("stock_summary_final.xlsx")


2025-02-04 20:54:48.515060+09:00
0 위메이드_0
2025-02-04 20:54:48.516223+09:00
112040
2025-02-04 20:54:49.238528+09:00
       시가     고가     저가     종가     거래량       등락률
0   39050  39700  38400  39000  317756 -0.889454
1   39000  39350  37250  38900  283767 -0.256410
2   38000  38850  37350  38400  154880 -1.285347
3   38100  38150  36450  36650  280210 -4.557292
4   36950  40800  36400  40100  547754  9.413370
..    ...    ...    ...    ...     ...       ...
56  38250  38800  37900  38200  131555  0.924703
57  38500  39100  37550  38150  201570 -0.130890
58  38200  40100  37600  38400  361159  0.655308
59  38800  40800  38400  39950  398439  4.036458
60  40500  40500  39800  40000  144625  0.125156

[61 rows x 6 columns]
2025-02-04 20:54:50.349973+09:00
1 RFHIC_1
2025-02-04 20:54:50.352675+09:00
218410
2025-02-04 20:54:52.412534+09:00
       시가     고가     저가     종가     거래량        등락률
0   14770  14830  13710  14380  218883  -1.708817
1   14400  14600  14230  14390  148878   0.069541
2   1438

In [25]:
import pandas as pd
from pykrx import stock
from datetime import datetime
import multiprocessing as mp
from functools import lru_cache
import pytz

seoul_tz = pytz.timezone('Asia/Seoul')

@lru_cache(maxsize=128)
def cached_get_ticker_list(date):
    return stock.get_market_ticker_list(date, "ALL")

def process_stock(stock_info, idx):
    company_name = f"{stock_info['종목명']}_{idx}"
    start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
    end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

    print(f"Processing {idx}: {company_name}")

    tickers = cached_get_ticker_list(start_date)
    ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

    if ticker is None:
        print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
        return None

    df = stock.get_market_ohlcv(start_date, end_date, ticker)

    if df.empty:
        print(f"데이터가 없습니다: {stock_info['종목명']}")
        return None

    start_price = df['종가'].iloc[0]
    end_price = df['종가'].iloc[-1]
    max_price = df['종가'].max()
    min_price = df['종가'].min()

    return {
        'company': company_name,
        'start_price': start_price,
        'end_price': end_price,
        'min_price': min_price,
        'max_price': max_price,
        'min_rate': round(((min_price / start_price) - 1) * 100, 2),
        'max_rate': round(((max_price / start_price) - 1) * 100, 2),
        'min_date': df['종가'].idxmin(),
        'max_date': df['종가'].idxmax(),
        'final_rate': round(((end_price / start_price) - 1) * 100, 2)
    }

def get_stock_data_summary(stock_list):
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.starmap(process_stock, [(s, i) for i, s in enumerate(stock_list)])

    results = [r for r in results if r is not None]

    if not results:
        return pd.DataFrame()

    df = pd.DataFrame(results)
    df.set_index('company', inplace=True)

    return df.T

if __name__ == "__main__":
    start_time = datetime.now(seoul_tz)
    print(f"시작 시간: {start_time}")

    stocks = [
        {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
        {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
        {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
        {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
        {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
    ]

    result_df = get_stock_data_summary(stocks)
    print(result_df)

    end_time = datetime.now(seoul_tz)
    print(f"종료 시간: {end_time}")
    print(f"총 소요 시간: {end_time - start_time}")

    result_df.to_excel("stock_summary_final.xlsx")


시작 시간: 2025-02-04 21:02:53.304585+09:00
Processing 1: RFHIC_1
Processing 0: 위메이드_0
Processing 2: 야스_2
Processing 3: 두산_3
Processing 4: 코오롱티슈진_4
company                   위메이드_0              RFHIC_1                 야스_2  \
start_price                39000                14380                10540   
end_price                  40000                14810                 8390   
min_price                  34850                10650                 7540   
max_price                  46400                14900                10710   
min_rate                  -10.64               -25.94               -28.46   
max_rate                   18.97                 3.62                 1.61   
min_date     2025-01-02 00:00:00  2024-12-09 00:00:00  2024-12-27 00:00:00   
max_date     2024-12-05 00:00:00  2025-01-21 00:00:00  2024-11-18 00:00:00   
final_rate                  2.56                 2.99                -20.4   

company                     두산_3             코오롱티슈진_4  
start_price        